In [ ]:
!cp "/content/drive/MyDrive/python/final_dataset.tar" "/content/data/final_dataset.tar"

In [ ]:
!tar -xvf "/content/data/final_dataset.tar"

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
People/072517_3M39.JPG
People/072517_4M39.JPG
People/072517_5M40.JPG
People/072517_6M40.JPG
People/072517_7M35.JPG
People/072517_8M40.JPG
People/072517_9M40.JPG
People/072540_0F37.JPG
People/072540_1F38.JPG
People/072540_2F38.JPG
People/072547_0M39.JPG
People/072547_1M41.JPG
People/072547_2M40.JPG
People/072547_3M40.JPG
People/072547_4M40.JPG
People/072547_5M41.JPG
People/072555_0M40.JPG
People/072555_1M41.JPG
People/072569_0M38.JPG
People/072569_1M39.JPG
People/072575_0M37.JPG
People/072575_1M37.JPG
People/072575_2M38.JPG
People/072594_03M41.JPG
People/072594_04M42.JPG
People/072594_0M38.JPG
People/072594_1M38.JPG
People/072594_2M40.JPG
People/072597_0F43.JPG
People/072597_1F44.JPG
People/072624_0M38.JPG
People/072624_1M38.JPG
People/072624_2M39.JPG
People/072624_3M39.JPG
People/072646_0M46.JPG
People/072646_1M47.JPG
People/072646_2M47.JPG
People/072646_3M48.JPG
People/072646_4M48.JPG
People/072646_5M49.JPG
People/072652_00M53.JPG
People/072652_01M5

In [ ]:
!cp "/content/drive/MyDrive/python/praticec_model.py" "/content/model/praticec_model.py"

In [ ]:
# -*- coding:utf-8 -*-
from random import shuffle
from model.praticec_model import *

import tensorflow as tf
import numpy as np
import os
import easydict

FLAGS = easydict.EasyDict({"animal_path": "/content/Animal",

                           "people_path": "/content/People",
                           
                           "num_classes": 3,

                           "lr": 0.0001,
                           
                           "img_size": 224,
                           
                           "batch_size": 16})

optim = tf.keras.optimizers.Adam(FLAGS.lr)

def input_func(input_path, label):
    # ["path/img.jpg", "1"]
    img = tf.io.read_file(input_path) # raw한 이미지 데이터가 저장된다
    img = tf.image.decode_jpeg(img, 3)  # raw한 이미지 데이터를 jpeg로 디코딩해준것(불명확했던 확장자를 확정해주는것)
    img = tf.image.resize(img, [FLAGS.img_size, FLAGS.img_size]) / 255. # 전체 이미지를 동일한 사이즈 (CNN의 입력으로 사용해야하기 때문에)로 만들어주기위한 전처리

    lab = tf.one_hot(label, FLAGS.num_classes)


    # 만일 3개의 클래스가 있다고 가정하자
    # 0, 1, 2
    # 이를 원핫 벡터 즉, 스칼라 값을 행렬로 만들어주는것을 의미한다.
    # 0 --> [1, 0, 0]
    # 1 --> [0, 1, 0]
    # 2 --> [0, 0, 1]

    # 5 - 5, 6 - 5, 7 - 5 = 0, 1, 2
    # 5 --> [1, 0, 0]

    return img, lab

def test_func(input_path, label):
    img = tf.io.read_file(input_path) # raw한 이미지 데이터가 저장된다
    img = tf.image.decode_jpeg(img, 3)  # raw한 이미지 데이터를 jpeg로 디코딩해준것(불명확했던 확장자를 확정해주는것)
    img = tf.image.resize(img, [FLAGS.img_size, FLAGS.img_size]) / 255. # 전체 이미지를 동일한 사이즈 (CNN의 입력으로 사용해야하기 때문에)로 만들어주기위한 전처리

    return img, label

def cal_loss(model, images, labels):

    # loss를 계산함
    # 딥러닝은 학습한다라고 했을 때, 학습의 의미는 weight를 새롭게 갱신하는것
    # weight를 새롭게 갱신할 떄 필요한것 --> back propagation --> weight를 (편)미분 시키는것 --> 기울기 --> gradient
    # 실질적으로 weight를 갱신하는것은 gradient와 optimzation이다.
    # update W = pervious W - lr * grad(W) --> SGD
    with tf.GradientTape() as tape: # with 문 -> 강제 조건문
        
        logits = model(images, training=True)   # logits은 이 모델의 마지막 output, model(images, training=True) --> model의 입력을 images로 하고 학습은 True로 하겠다
        # logits shape--> [16, 3]   # 사람 고양이 강아지   [x, y, z]
        # labels shape --> [16, 3]  # 사람(0) 고양이(1) 강아지(2) --> 사람[1, 0, 0] 고양이[0, 1, 0] 강아지[0, 0, 1] 

        loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True)(labels, logits)    # logits => [x, y, z]가 16개 겹쳐있음
        # [0, 1] --> 0(사건)이 일어날 확률 0.5, 1이 발생할 확률 0.5; [0,10] --> 을 softmax를 하면? [0 , 0.9]
        # from_logits=True의 의미는 마지막 CNN output에 즉, logit에 softmax activation을 추가한것

    grads = tape.gradient(loss, model.trainable_variables)  # 편미분을 구했으니, 이제는 편미분 한것을 optimzation에 적용
    optim.apply_gradients(zip(grads, model.trainable_variables))    # c = zip(a,b) --> a = [1,2,3] b = [3,4,5]; c = [[1,3], [2,4], [3, 5]]

    return loss

def cal_acc(model, images, labels):
    # 정확도를 측정한다는것은, 정답이 무조건적으로 존재할때 측정할 수 있는거죠?

    # 1. model의 출력이 필요하다
    logits = model(images, False)   # [1, 3]--> 2차원
    logits = tf.squeeze(logits, 0) # 차원중에 1이 존재할때, 1이라는 차원을 제거시켜주는 함수    [3,]
    predict = tf.nn.softmax(logits) # [3,] --> [x일 확률, y일 확률, z일 확률]
    predict = tf.argmax(predict, output_type=tf.int32)    # 버퍼에 존재하는 값들중에 가장 큰 값을 차지하는 index를 반환시켜주는 함수

    ground_truth = labels[0] # [1,] --> []
    acc = tf.equal(predict, ground_truth)   # a 와 b가 같으면 True (1)로 반환, 다르면 False (0)로 반환; shape = [] 단일 bool형 
    acc = tf.cast(acc, tf.float32)  # [True] --> [1.], [False] --> [0.]

    return acc  # 값이 하나만 반환됨

def main():
    # 여기서 우선적으로 모델을 불러와야한다. --> practive_model에 있는것을 불러오면 된다.
    model = modified_model(input_shape=(FLAGS.img_size, FLAGS.img_size, 3))
    model.summary()

    # People - 0, Cat - 1, Dog - 2 labeling, 3가지 클래스(label, ground truth, 정답 지표) 에 대해서 분류하기 위해 작업한것
    # Convolutional Neural Network (CNN) - 분류 
    animal_data = os.listdir(FLAGS.animal_path)    # C:/Users/Yuhwan/Downloads/final_dataset + "/" + Animal"
    people_data = os.listdir(FLAGS.people_path)    # poeple data에는 오직 파일 이름 존재한다. 그렇기에 경로까지 같이 포함시켜
    # 줘야한다.
    
    people_img = [FLAGS.people_path + "/" + img for img in people_data]    # 경로와 사람 이름이 포함된 리스트들이 저장이됨
    people_lab = [0 for i in range(len(people_img))]    # people_data 개수만큼 라벨을 만든것이다.

    Cat_data = []
    cat_lab = []
    Dog_data = []
    dog_lab = []
    for img in animal_data:
        name = img.split('_')  # name은 Cat or Dog string type 을 저장시킬것   ['', ''] Cat_10.jpg
        if name[0] == "Cat":
            Cat_data.append(img)
            cat_lab.append(1)
        if name[0] == "Dog":
            Dog_data.append(img)
            dog_lab.append(2)

    cat_img = [FLAGS.animal_path + '/' + img for img in Cat_data]  # python list [경로/고양이1, 경로/고양이2,.....]
    dog_img = [FLAGS.animal_path + "/" + img for img in Dog_data]
    # Cat, Dog or People에 대한 버퍼는 문자열이 저장된거에요 --> 메모리에 저장되는 공간을 많이 잡아먹지않는다.
    # 코딩을 효율적으로 할 수 있다.

    # 여러분들이 찾아보면 알겠지만, 보통 튜토리얼로 만들어진 데이터들은
    # 실제 이미지 값들이 저장됩니다. --> 메모리를 많이 잡아먹는다 --> cpu 메모리때문에 학습이 안된다. --> 

    # tensorflow를 하려면 우선적으로 numpy array 가 필요하다.
    # 그러면 python list를 numpy array로 만들어주면 된다
    
    train_img = []
    train_lab = []
    test_img = []
    test_lab = []
    for i in range(len(people_img) // 2):
        train_img.append(people_img[i])
        train_lab.append(people_lab[i])
    for i in range(len(people_img) // 2, len(people_img)):
        test_img.append(people_img[i])
        test_lab.append(people_lab[i])

    for i in range(len(cat_img) // 2):
        train_img.append(cat_img[i])
        train_lab.append(cat_lab[i])
    for i in range(len(cat_img) // 2, len(cat_img)):
        test_img.append(cat_img[i])
        test_lab.append(cat_lab[i])

    for i in range(len(dog_img) // 2):
        train_img.append(dog_img[i])
        train_lab.append(dog_lab[i])
    for i in range(len(dog_img) // 2, len(dog_img)):
        test_img.append(dog_img[i])
        test_lab.append(dog_lab[i])

    #########################################
    A = list(zip(train_img, train_lab))
    shuffle(A)
    train_img, train_lab = zip(*A)
    #########################################

    train_img = np.array(train_img, dtype=np.str)
    train_lab = np.array(train_lab, dtype=np.int32)
    test_img = np.array(test_img, dtype=np.str)
    test_lab = np.array(test_lab, dtype=np.int32)
    count = 0

    test_gener = tf.data.Dataset.from_tensor_slices((test_img,test_lab))
    test_gener = test_gener.map(test_func)
    test_gener = test_gener.batch(1)
    test_gener = test_gener.prefetch(tf.data.experimental.AUTOTUNE)

    for epoch in range(1):
        train_gener = tf.data.Dataset.from_tensor_slices((train_img,train_lab))
        train_gener = train_gener.map(input_func)
        train_gener = train_gener.batch(FLAGS.batch_size)   # (동영상 말고 사진) CNN 입력은 무조건 한 이미지씩 진행된ㄴ겁니다.
        train_gener = train_gener.prefetch(tf.data.experimental.AUTOTUNE)

        train_iter = iter(train_gener)  # 배치단위로 묶인 이미지가 저장되어있는 버퍼의 성분의 총 개수 = (총 이미지 개수 // 배치) 
        train_range = len(train_img) // FLAGS.batch_size
        for step in range(train_range):
            img, lab = next(train_iter) # img - [batch, height, width, channel] 4 차원 행렬

            loss = cal_loss(model, img, lab)

            if count % 10 ==0: # count가 10의 배수일때 이 조건문이 작동함
                print("Epoch = {} [{}/{}] loss = {}".format(epoch, step + 1, train_range, loss))

            if count % 100 == 0 and count != 0:
                test_iter = iter(test_gener)
                sum = 0.
                for i in range(len(test_img)):
                    test_images, test_labels = next(test_iter)
                    sum += cal_acc(model, test_images, test_labels)
                print("Accuracy = {}% (step = {})".format((sum / len(test_img)) * 100., count))


            count += 1  # total step


if __name__ == "__main__":
    main()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 224, 224, 64)      1728      
_________________________________________________________________
batch_normalization_5 (Batch (None, 224, 224, 64)      256       
_________________________________________________________________
re_lu_5 (ReLU)               (None, 224, 224, 64)      0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 112, 112, 128)     73728     
_________________________________________________________________
batch_normalization_6 (Batch (None, 112, 112, 128)     512       
_________________________________________________________________
re_lu_6 (ReLU)               (None, 112, 112, 128)     0   